In [1]:
import requests
import json
import pandas as pd

In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
lfm_key = os.getenv("lastfm_key")

In [6]:
cosaslista = pd.read_csv("technodatabase.csv", index_col=0)

In [7]:
columnanombreartista = cosaslista["nombre_artista"].tolist()

In [8]:
type(columnanombreartista)
len(columnanombreartista)

1250

In [9]:
def llamar_api(url):
    llamada = requests.get(url)
    
    if llamada.status_code != 200:
        print(llamada.reason)
    else:
        return llamada.json()

In [11]:
infoartistas={"nombre_artista":[],"bio":[],"listeners":[],"playcount":[],"similar":[]}

for nombreartista in columnanombreartista:
    
    if nombreartista == "Various Artists":  # elimiamos Various artists para q no salgan datos erróneos en las bio
        continue

    elif nombreartista in infoartistas["nombre_artista"]: 
        continue  # Skip this iteration if the artist is a duplicate 

    else:    
        try: 
            url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={nombreartista}&api_key={lfm_key}&format=json"
            respuesta = llamar_api(url)

            #Para hacer esto: infoartistas["similar"].append(respuesta["artist"]["similar"]["artist"][0]['name'])
            similares=[]
            for artistaejemplo in respuesta["artist"]["similar"]["artist"]:
                similares.append(artistaejemplo['name'])

            infoartistas["similar"].append(similares)
            infoartistas["nombre_artista"].append(nombreartista)
            infoartistas["bio"].append(respuesta["artist"]["bio"]["summary"])
            infoartistas["listeners"].append(respuesta["artist"]["stats"]["listeners"])
            infoartistas["playcount"].append(respuesta["artist"]["stats"]["playcount"])

            

        except KeyError:
            continue 

print(infoartistas)

{'nombre_artista': ['Disco Van', 'Niño Árbol', 'Alexander Kowalski', 'Nineteen Sines', 'DP-6', 'Norberto Lusso', 'Marco Ginelli', 'SUBSET', 'Mila Journée', 'Markus Schulz - Global DJ Broadcast', 'Luix Spectrum', 'Sascha Audit', 'Manel Diaz', 'Greg Denbosa', 'Dkult', 'Duck Sandoval', 'Darknezz', 'Fabrice Torricella', 'Maguta', 'Marc Molina', 'Apart', 'Christian Gerlach', 'Deeperwalk', 'I7HVN', 'Melodymann', 'Lorenzo Al Dino', 'Matteo Vitanza', 'R3ckzet', 'Vincent Noxx', 'Padhme', 'Droneghost', 'TML', 'LetKolben', 'AAAA', 'Dj Scale Ripper', 'I Am Bam', 'Nae:Tek', 'Danilenko', 'Modular Phaze', 'Primary Substance', 'Yan Oxygen', 'Israel Toledo', 'FU-5', 'Mari Mattham', 'Darkrow', 'Austrek', 'Tender H', 'Omega Drive', 'C-System', 'Ignacio Arfeli', 'Lewis G. Burton', 'Chiraya', 'Aves Volare', 'Dave Wesley', 'Namito', 'David Durst', 'Guy Mantzur', 'Seventh Sea', 'Sven Kuhlmann', 'Crimson Butterfly', 'Lady Maru', 'Sopik', 'Kinko', 'Isabassi', 'Sergy Casttle', 'Mown', 'Samantha Togni', 'Denizen

In [12]:
df=pd.DataFrame(infoartistas)

In [13]:
df

,nombre_artista,bio,listeners,playcount,similar
0,Disco Van,"<a href=""https://www.last.fm/music/Disco+Van""...",25,258,[]
1,Niño Árbol,Kevin Martinez is part of a new generation of ...,2020,6871,"[Cabizbajo, El Irreal Veintiuno, Camille Mando..."
2,Alexander Kowalski,"Kowalski was born in 1978 in Greifswald, Germa...",86224,598784,"[Heiko Laux, Anthony Rother, Jeff Mills, Johan..."
3,Nineteen Sines,"<a href=""https://www.last.fm/music/Nineteen+S...",497,1197,"[Niereich, Lilly Palmer, Abyssvm, Tim Tama, Cl..."
4,DP-6,DP-6 project's participants are Alexey Filin a...,7275,36833,"[East Cafe, Platunoff, Monojoke, Blusoul, Mich..."
...,...,...,...,...,...
455,Alveol,Sweden's Ludvig Cimbrelius has been developing...,4551,67879,"[Yone b, Kogyo, Fingers in the Noise, Subforms..."
456,Serge Landar,"<a href=""https://www.last.fm/music/Serge+Land...",991,6904,"[Tim Penner, Platunoff, Michael A, Ewan Rill, ..."
457,Gareth Wild,"<a href=""https://www.last.fm/music/Gareth+Wil...",2552,10454,"[Endlec, Subjected, Plukkk, D. Carbone, Exium]"
458,mja music switzerland,"<a href=""https://www.last.fm/music/mja+music+...",5,13,[]


In [15]:
df.to_csv("techno_lastfmdatabase.csv")